### 주제: 5대 리그 (PL, LaLiga, Serie A, Bundesliga, Ligue 1) 내 이적시장 목록 크롤링 및 자동화

In [35]:
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
    }

In [245]:
league_code = ['GB1', 'ES1', 'IT1', 'L1', 'FR1']    # PL, LaLiga, Serie A, Bundesliga, Ligue 1
name = []
position = []
age = []
nation = []
left = []
joined = []
transfer_date = []
market_value = []
fee = []

for league in league_code:
    url = f'https://www.transfermarkt.com/transfers/neuestetransfers/statistik/plus/plus/1/galerie/0/wettbewerb_id/{league}/land_id//minMarktwert/500.000/maxMarktwert/500.000.000/minAbloese/0/maxAbloese/500.000.000/yt0/Show/page/1?ajax=yw1'
    response = requests.get(url, headers=headers)
    # print(response.text)
    soup = BeautifulSoup(response.text, 'html.parser')
    # transfer_lists = soup.find_all('tr',{'class':['odd','even']})
    transfer_lists = soup.select('tr.odd, tr.even')
    # print(transfer_lists)
    
    for transfer_list in transfer_lists:
        info = transfer_list.select('td')
        # for i in enumerate(info):
        #     print(i)
        name.append(info[2].text.strip())
        position.append(info[3].text.strip())
        age.append(info[4].text.strip())
        nation.append(info[5].img['alt'])
        left.append(info[6].img['alt'])
        joined.append(info[10].img['alt'])
        transfer_date.append(info[14].text.strip())
        market_value.append(info[15].text.strip())
        fee.append(info[16].text.strip())


In [246]:
print(name)
print(position)
print(age)
print(nation)
print(left)
print(joined)
print(transfer_date)
print(market_value)
print(fee)

['Richie Laryea', 'Manuel Lanzini', 'Luke McNally', 'Aaron Connolly', 'Deniz Undav', 'Lucas Moura', 'Braian Ojeda', 'Josh Bowler', 'Lesley Ugochukwu', 'Iliman Ndiaye', 'Luke Plange', 'Jack Colback', 'Fabinho', 'Carl Rushworth', 'Allan Saint-Maximin', 'Karl Darlow', 'Wesley Moraes', 'Ryan Giles', 'Calvin Bassey', 'Fred Onyedinma', 'Riyad Mahrez', 'Jamal Lewis', 'Ionut Radu', 'Jordan Henderson', 'Jacob Bruun Larsen', 'Rony Lopes', 'Mario Hernández', 'André Silva', 'Erick Cabaco', 'Julián Araujo', 'Diego Lainez', 'Álex Collado', 'Jorge Meré', 'Florin Andone', 'Rubén Blanco', 'Karim Rekik', 'Munir El Haddadi', 'Nico González', 'Edinson Cavani', 'El Bilal Touré', 'Florian Lejeune', 'Rafa Marín', 'Gerard Gumbau', 'Samuel Chukwueze', 'Álex Collado', 'Chadi Riad', 'Johan Mojica', 'Isco', 'Alberto Marí', 'Matteo Gabbia', "Danilo D'Ambrosio", 'Mert Müldür', 'Miguel Veloso', 'Morten Thorsby', 'Magnus Warming', 'Marco Carraro', 'Giuseppe Ambrosino', 'Stefano Gori', 'Matías Viña', 'Benoît Costil', 

In [315]:
df = pd.DataFrame(
    {'Name': name,
    'Position': position,
    'Age': age,
    'Nation': nation,
    'Left': left,
    'Joined': joined,
    'Transfer Date': transfer_date,
    'Market Value': market_value,
    'Fee': fee
})

df

,Name,Position,Age,Nation,Left,Joined,Transfer Date,Market Value,Fee
0,Richie Laryea,Right-Back,28,Canada,Nottingham Forest,Vancouver Whitecaps FC,"Aug 2, 2023",€2.50m,loan transfer
1,Manuel Lanzini,Attacking Midfield,30,Argentina,West Ham United,CA River Plate,"Aug 2, 2023",€6.00m,free transfer
2,Luke McNally,Centre-Back,23,Ireland,Burnley FC,Stoke City,"Aug 2, 2023",€2.20m,loan transfer
3,Aaron Connolly,Centre-Forward,23,Ireland,Brighton & Hove Albion,Hull City,"Aug 2, 2023",€4.00m,?
4,Deniz Undav,Centre-Forward,27,Germany,Brighton & Hove Albion,VfB Stuttgart,"Aug 2, 2023",€10.00m,Loan fee:€700k
...,...,...,...,...,...,...,...,...,...
120,Papa Ndiaga Yade,Left Winger,23,Senegal,FC Metz,Quevilly Rouen Métropole,"Jul 27, 2023",€1.20m,loan transfer
121,Bradley Locko,Left-Back,21,France,Stade Reims,Stade Brestois 29,"Jul 27, 2023",€2.50m,€500k
122,Maxime Colin,Right-Back,31,France,Birmingham City,FC Metz,"Jul 27, 2023",€1.20m,free transfer
123,Rafael Ratão,Left Winger,27,Brazil,FC Toulouse,Esporte Clube Bahia,"Jul 26, 2023",€2.50m,€2.50m


### 데이터 전처리

In [317]:
# 5대 리그 사이에서 이적이 발생할 경우, 중복 발생하므로 중복 제거
df.drop_duplicates(['Name'], inplace=True)
df.reset_index(inplace=True, drop=True)

In [318]:
# Age 컬럼 정수형으로 변환
df['Age'] = df['Age'].astype('int64')

In [319]:
# 날짜 형식 변환
df['Transfer Date'] = pd.to_datetime(df['Transfer Date'])

In [320]:
# 임대/이적 구분
df['Transfer Type'] = df['Fee'].apply(lambda x: 'loan' if re.search('[lL]oan.+', x) else 'unknown' if x == 'unknown' else 'permanent transfer')

In [321]:
# 시장 가치 데이터 변환
df['Market Value'] = df['Market Value'].apply(lambda x: re.search('[0-9\.]{1,}[mk]', x).group() if re.search('[0-9\.]{1,}[mk]', x) else 'unknown' if x == '?' else 0)
df['Market Value'] = df['Market Value'].apply(lambda x: float(str(x)[:-1])*10e6 if str(x)[-1] == 'm' else float(str(x)[:-1])*10e3 if str(x)[-1] == 'k' else 'unknown' if str(x) == 'unknown' else 0)

In [322]:
# 비용 데이터 변환
df['Fee'] = df['Fee'].apply(lambda x: re.search('[0-9\.]{1,}[mk]', x).group() if re.search('[0-9\.]{1,}[mk]', x) else 'unknown' if x == '?' else 0)
df['Fee'] = df['Fee'].apply(lambda x: float(str(x)[:-1])*10e6 if str(x)[-1] == 'm' else float(str(x)[:-1])*10e3 if str(x)[-1] == 'k' else 'unknown' if str(x) == 'unknown' else 0)

In [323]:
# Fee column명 변경
df.rename(columns={'Market Value': 'Market Value (€)', 'Fee': 'Fee (€)'}, inplace=True)

In [339]:
df

,Name,Position,Age,Nation,Left,Joined,Transfer Date,Market Value (€),Fee (€),Transfer Type
0,Richie Laryea,Right-Back,28,Canada,Nottingham Forest,Vancouver Whitecaps FC,2023-08-02,25000000.0,0,loan
1,Manuel Lanzini,Attacking Midfield,30,Argentina,West Ham United,CA River Plate,2023-08-02,60000000.0,0,permanent transfer
2,Luke McNally,Centre-Back,23,Ireland,Burnley FC,Stoke City,2023-08-02,22000000.0,0,loan
3,Aaron Connolly,Centre-Forward,23,Ireland,Brighton & Hove Albion,Hull City,2023-08-02,40000000.0,unknown,permanent transfer
4,Deniz Undav,Centre-Forward,27,Germany,Brighton & Hove Albion,VfB Stuttgart,2023-08-02,100000000.0,7000000.0,loan
...,...,...,...,...,...,...,...,...,...,...
106,Dario Maresic,Centre-Back,23,Austria,Stade Reims,NK Istra 1961,2023-07-27,5000000.0,unknown,permanent transfer
107,Papa Ndiaga Yade,Left Winger,23,Senegal,FC Metz,Quevilly Rouen Métropole,2023-07-27,12000000.0,0,loan
108,Bradley Locko,Left-Back,21,France,Stade Reims,Stade Brestois 29,2023-07-27,25000000.0,5000000.0,permanent transfer
109,Maxime Colin,Right-Back,31,France,Birmingham City,FC Metz,2023-07-27,12000000.0,0,permanent transfer
